In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

ROOT_PATH='...'

In [ ]:
# pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_colwidth', None)

# Getting scraping information

In [ ]:
scraper_csv_path = ROOT_PATH+'/resources/data/raw/scraping.csv'
s_data_df  = pd.read_csv(scraper_csv_path,sep=',',encoding='utf-8')  # header=None,skiprows=[0]
s_data_df.drop(['web-scraper-order', 'web-scraper-start-url', 'web-scraper-start-url', 'ofertalink-href', 'page', 'page-href', 'ofertalink'], axis=1, inplace=True)

s_data_df['numerooferta'] = s_data_df['numerooferta'].astype(str).str.replace('Datos de la oferta número:', '')

s_data_df['contract'] = s_data_df['contract'].astype(str).str.replace('Duración del contrato:', '')
s_data_df['contract'] = s_data_df['contract'].astype(str).str.strip()

s_data_df['cityandprovince'] = s_data_df['cityandprovince'].astype(str).str.replace('Localidad de Ubicación del Puesto:', '')
s_data_df['cityandprovince'] = s_data_df['cityandprovince'].astype(str).str.replace('\n', '')
s_data_df['cityandprovince'] = s_data_df['cityandprovince'].astype(str).str.replace('\t', '')
s_data_df['cityandprovince'] = s_data_df['cityandprovince'].astype(str).str.strip()

s_data_df['datosadicionales'] = s_data_df['datosadicionales'].astype(str).str.replace('\n', '')
s_data_df['datosadicionales'] = s_data_df['datosadicionales'].astype(str).str.replace('\t', '')
s_data_df['datosadicionales'] = s_data_df['datosadicionales'].astype(str).str.strip()

s_data_df['contract'] = s_data_df['contract'].astype(str).str.replace('días', '')
s_data_df['contract'] = s_data_df['contract'].astype(str).str.strip()

s_data_df['fecha'] = s_data_df['fecha'].astype(str).str.replace('Fecha de inicio:', '')
s_data_df['fecha'] = s_data_df['fecha'].astype(str).str.strip()

s_data_df['province'] = s_data_df['province'].astype(str).str.replace('Provincia:', '')
s_data_df['province'] = s_data_df['province'].astype(str).str.strip()

print('Number of recovered records: ', s_data_df.shape[0])

s_data_df = s_data_df.sort_values('numerooferta')

#printing all duplicates
col_list = s_data_df.columns.tolist()
s_data_df = s_data_df.replace('nan',np.nan,regex=True).groupby('numerooferta').apply(lambda x : x.ffill().bfill())

s_data_df = s_data_df.drop_duplicates(subset=['numerooferta'], keep='first')

s_data_df[['city','province_aux']] = s_data_df['cityandprovince'].astype(str).str.split("(", expand=True,)

s_data_df['province_aux'] = s_data_df['province_aux'].astype(str).str.replace(')', '')
s_data_df['province_aux'] = s_data_df['province_aux'].astype(str).str.strip()

s_data_df.drop(['cityandprovince', 'province_aux'], axis=1, inplace=True)

s_data_df.rename(columns={'numerooferta':'id_job', 'datosadicionales': 'description_additional', 'nivelprofesionalbuscado': 'professional_level', 'requerimientos': 'requirement', 'contract': 'contract_duration', 'oferta': 'job', 'fecha':'date'}, inplace=True)

print('Number of preprocessed records: ', s_data_df.shape[0])
display(s_data_df.head(3))

# Getting PDF information

In [ ]:
file_csv_path = ROOT_PATH+'/resources/data/cleaned/df_data_pdf.csv'
data_df  = pd.read_csv(file_csv_path,sep=',',encoding='utf-8')  # header=None,skiprows=[0]
data_df.drop(['english_level'], axis=1, inplace=True)
data_df.drop(['number'], axis=1, inplace=True)

print('Number of rows: ', data_df.shape[0])
data_df.head(3)

# Mixing both df

In [ ]:
s_data_df['id_job']=s_data_df['id_job'].astype(int)
unified_data_df = pd.merge(data_df, s_data_df, on=['id_job', 'job', 'date'], how='outer')
unified_data_df = unified_data_df.sort_values('province_x')

display(unified_data_df.head(3))
print(unified_data_df.shape)

In [ ]:
unified_data_df.to_csv(ROOT_PATH+'/resources/data/cleaned/df_data_unified.csv', index=False)

# Preprocessing unified data

In [ ]:
df_data_unified = pd.read_excel(ROOT_PATH+'/src/main/nb/df_data_unified.xlsx')  
display(df_data_unified.head(3))

pd.set_option('display.max_rows', None)

In [ ]:
# City to upper case:
df_data_unified['city'] = df_data_unified['city'].astype(str).str.upper()
df_data_unified['city']=df_data_unified['city'].astype(str).str.replace('ALAGON','ALAGÓN')
df_data_unified['city']=df_data_unified['city'].astype(str).str.replace('ALMUDEVAR','ALMUDÉVAR')
df_data_unified['city']=df_data_unified['city'].astype(str).str.replace('ALMUNIA DE DOÑA GODINA, LA','LA ALMUNIA DE DOÑA GODINA')
df_data_unified['city']=df_data_unified['city'].astype(str).str.replace('ALQUEZAR','ALQUÉZAR')
df_data_unified['city']=df_data_unified['city'].astype(str).str.replace('ANDORRA,TERUEL','ANDORRA, TERUEL')
df_data_unified['city']=df_data_unified['city'].astype(str).str.replace('BINEFAR','BINÉFAR')
df_data_unified['city']=df_data_unified['city'].astype(str).str.replace('BURGO DE EBRO, EL','EL BURGO DE EBRO')
df_data_unified['city']=df_data_unified['city'].astype(str).str.replace('GINEBROSA, LA','LA GINEBROSA')
df_data_unified['city']=df_data_unified['city'].astype(str).str.replace('GRAÑEN','GRAÑÉN')
df_data_unified['city']=df_data_unified['city'].astype(str).str.replace('GRADO, EL','EL GRADO')
df_data_unified['city']=df_data_unified['city'].astype(str).str.replace('IGRIES','IGRIÉS')
df_data_unified['city']=df_data_unified['city'].astype(str).str.replace('JOYOSA, LA','LA JOYOSA')
df_data_unified['city']=df_data_unified['city'].astype(str).str.replace('MAGALLON','MAGALLÓN')
df_data_unified['city']=df_data_unified['city'].astype(str).str.replace('MARIA DE HUERVA','MARÍA DE HUERVA')
df_data_unified['city']=df_data_unified['city'].astype(str).str.replace('MAZALEON','MAZALEÓN')
df_data_unified['city']=df_data_unified['city'].astype(str).str.replace('MONZON','MONZÓN')
df_data_unified['city']=df_data_unified['city'].astype(str).str.replace('MAZALEON','MAZALEÓN')
df_data_unified['city']=df_data_unified['city'].astype(str).str.replace('PUEBLA DE ALFINDEN, LA','LA PUEBLA DE ALFINDEN')
df_data_unified['city']=df_data_unified['city'].astype(str).str.replace('PUEBLA DE VALVERDE, LA','LA PUEBLA DE VALVERDE')
df_data_unified['city']=df_data_unified['city'].astype(str).str.replace('SABIÑANIGO','SABIÑÁNIGO')
df_data_unified['city']=df_data_unified['city'].astype(str).str.replace('SARRION','SARRIÓN')
df_data_unified['city']=df_data_unified['city'].astype(str).str.replace('SOTONERA, LA','LA SOTONERA')
df_data_unified['city']=df_data_unified['city'].astype(str).str.replace('VILLANUEVA DE GALLEGO','VILLANUEVA DE GÁLLEGO')
df_data_unified['city']=df_data_unified['city'].astype(str).str.replace('VILLANUA','VILLANÚA')
df_data_unified['city']=df_data_unified['city'].astype(str).str.replace('EPILA','ÉPILA')

df_data_unified.groupby(['city']).size().reset_index(name='count')

In [ ]:
# Office:
df_data_unified['office']=df_data_unified['office'].astype(str).str.replace('SABIÑANIGO','SABIÑÁNIGO')
df_data_unified['office']=df_data_unified['office'].astype(str).str.replace('BINEFAR','BINÉFAR')
df_data_unified['office']=df_data_unified['office'].astype(str).str.replace('ZARAGOZA-SANTANDER','ZARAGOZA-CALLE SANTANDER')

df_data_unified.groupby(['office']).size().reset_index(name='count')

In [ ]:
# Province:
df_data_unified.groupby(['province']).size().reset_index(name='count')

In [ ]:
# Job:
df_data_unified['office']=df_data_unified['office'].astype(str).str.replace('ADMINISTRATIVOS-CONTABLES','ADMINISTRATIVOS CONTABLES')

df_data_unified.groupby(['job']).size().reset_index(name='count')

In [ ]:
# Sector:
df_data_unified.groupby(['sector']).size().reset_index(name='count')

In [ ]:
df_data_unified.to_csv(ROOT_PATH+'/src/main/nb/df_data_unified.csv', index=False)